In [1]:
import cv2

In [2]:
import os
import numpy as np

In [3]:
data_path = r"C:\Users\Siddharth\Desktop\logo\ammm\tmmm\dataset"

In [4]:
categories = os.listdir(data_path)

In [5]:
labels = [i for i in range (len(categories))]

In [6]:
label_dict = dict(zip(categories,labels))

In [7]:
print(label_dict)

{'with mask': 0, 'without mask': 1}


In [8]:
print(categories)

['with mask', 'without mask']


In [9]:
print(labels)

[0, 1]


In [10]:
img_size = 100

In [11]:
data=[]
target=[]
for category in categories:
    folder_path = os.path.join(data_path, category)
    img_names = os.listdir(folder_path)
    
    for img_name in img_names:
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
        
        try:
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            resized = cv2.resize(gray,(img_size,img_size))
            data.append(resized)
            target.append(label_dict[category])
        except Exception as e:
            print('Exception :',e)
                
    
           

In [12]:
data = np.array(data)/255.0
data = np.reshape(data,(data.shape[0],img_size,img_size,1))
target= np.array(target)

In [13]:
from keras.utils import np_utils
import tensorflow as tf

In [14]:
new_target = tf.keras.utils.to_categorical(target)

In [15]:
new_target


array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [16]:
np.save('data', data)
np.save('target', new_target )

In [17]:
##CNN

In [18]:
import numpy as np

In [19]:
data = np.load('data.npy')
target = np.load('target.npy')

In [20]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint

In [21]:
model = Sequential()

In [22]:
model.add(Conv2D(200,(3,3), input_shape = data.shape[1:] ))

In [23]:
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

In [24]:
model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

In [25]:
model.add(Flatten())
model.add(Dropout(0.5))

In [26]:
model.add(Dense(50, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics= ['accuracy'])


In [27]:
from sklearn.model_selection import train_test_split

In [28]:
train_data, test_data, train_target, test_target= train_test_split(data,target,test_size=0.1)

In [29]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.model', monitor = 'val_loss',verbose=0,save_best_only =True, mode='auto')

In [30]:
history= model.fit(train_data,train_target,epochs=20,callbacks=[checkpoint],validation_split=0.2)

Epoch 1/20
31/31 [==============================] - ETA: 0s - loss: 0.7114 - accuracy: 0.5515WARNING:tensorflow:From c:\users\siddharth\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\training\tracking\tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model-001.model\assets
31/31 [==============================] - 80s 3s/step - loss: 0.7114 - accuracy: 0.5515 - val_loss: 0.6800 - val_accuracy: 0.4879
Epoch 2/20
31/31 [==============================] - 80s 3s/step - loss: 0.6023 - accuracy: 0.6768 - val_loss: 0.5557 - val_accuracy: 0.6694
Epoch 3/20
31/31 [==============================] - 80s 3s/step - loss:

In [31]:
print(model.evaluate(test_data,test_target))

5/5 [==============================] - 2s 434ms/step - loss: 0.1127 - accuracy: 0.9565
[0.11267417669296265, 0.95652174949646]


In [32]:
from keras.models import load_model
import cv2

In [33]:
face_classifier = cv2.CascadeClassifier(r'C:\Users\Siddharth\Desktop\logo\ammm\tmmmharrcascade_frontalface_default.xml')

In [34]:
source = cv2.VideoCapture(0)

In [35]:
label_dict = {0:'Mask',1:'No Mask'}
coloe_dict = {0:(0,255,0),1:(0,0,255)}

In [36]:

while (True):
    ret, img = source.read()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces= face_classifier.detectMultiScale(gray,1.3,5)
    
    for x, y, w, h in faces:
        face_img = gray[y:y+w, x:x+w]
        resized= cv2.resize(face_img,(100,100))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,100,100,1))
        result= model.predict(reshaped)
        label= np.argmax(result,axis=1)[0]
        cv2.rectangle(img,(x,y),(x+w, y+h),coloe_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w, y),coloe_dict[label],-1)
        cv2.putText(img, label_dict[label],(x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        cv2.imshow('LIVE', img)
        key=cv2.waitKey(1)
        
        if(key==27):
           # break
   # cv2.desroyAllWindows()
    #source.release()